In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
import os
# Load variables from .env file into environment
load_dotenv()
from huggingface_hub import login
login(token=os.getenv("huggingFace_token"))


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\rajan\.cache\huggingface\token
Login successful


In [3]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV=os.getenv("PINECONE_API_ENV")

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("C:\\Users\\rajan\\OneDrive\\Desktop\\AI project\\Chatbot\\data")
documents=load_pdf("C:\\Users\\rajan\\OneDrive\\Desktop\\AI project\\Chatbot\\data")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [11]:

from pinecone import Pinecone
from langchain.vectorstores import Pinecone as PC

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medicalchatbot" 
index_name=index_name
index = pc.Index(index_name)  
# for i, t in zip(range(len(text_chunks)), text_chunks):
#    query_result = embeddings.embed_query(t.page_content)
#    index.upsert(
#    vectors=[
#         {
#             "id": str(i),  # Convert i to a string
#             "values": query_result, 
#             "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
#         }
#     ],
#     namespace="real" 
# )

# index.describe_index_stats() 


# Storing data in docsearch



In [12]:

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medicalchatbot" 
index_name=index_name
index = pc.Index(index_name) 
query="headache"
query_vector = embeddings.embed_query(query)
result=index.query(
    namespace="real",
    vector=query_vector,
    top_k=1,
    # include_values=True,
    include_metadata=True,
    # filter={"text":{"$eq":str(text_chunks[0].page_content)}}
)
print(result)


In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define the similarity search results
search_results =result


# Extract text from search results
texts = [match['metadata']['text'] for match in search_results['matches']]

# Concatenate text for model input
prompt = '\n'.join(texts)

# Tokenize input text
input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate response
output = model.generate(input_ids, max_length=len(input_ids[0]) + 50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Decode and print generated response
response = tokenizer.decode(output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
print(response)

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = CTransformers(model=r'C:\Users\rajan\OneDrive\Desktop\AI project\Chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin',
                    model_type="llama",
                    config={'max_new_tokens': 512, 'temperature': 0.8})


In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"
index = pc.Index(index_name)
from langchain_pinecone import PineconeVectorStore  
text_field = "text"  
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
pinecone = PineconeVectorStore.from_existing_index(index_name, embeddings)
 

In [ ]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=chain({"query": user_input})
#     print("Response : ", result["result"])
   

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pinecone import Pinecone

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"
index = pc.Index(index_name)

# Define the similarity search results (you need to provide 'result' variable)
search_results = result

# Extract text from search results
texts = [match['metadata']['text'] for match in search_results['matches']]

# Concatenate text for model input
prompt = '\n'.join(texts)

# Tokenize input text
input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate response
output = model.generate(input_ids, max_length=len(input_ids[0]) + 50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Decode and print generated response
response = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(response)

# Define the prompt template
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer: {answer}
"""

# Function to format and print the response
def print_formatted_response(context, question, answer):
    print(prompt_template.format(context=context, question=question, answer=answer))

# Function to handle user query
def handle_query(query):
    # Embed user query
    query_vector = embeddings.embed_query(query)
    # Query Pinecone index
    result = index.query(
        namespace="real",
        vector=query_vector,
        top_k=1,
        include_metadata=True
    )
    # Check if there are matches
    if result['matches']:
        # Extract context, question, and answer if available
        context = result['matches'][0]['metadata'].get('context', 'No context available')
        question = result['matches'][0]['metadata'].get('question', 'No question available')
        answer = result['matches'][0]['metadata'].get('answer', 'No answer available')
        # Print formatted response
        print_formatted_response(context, question, answer)
    else:
        print("No relevant information found for the query.")

# User input prompt
while True:
    user_query = input("Enter your question: ")
    # Handle user query
    handle_query(user_query)


Purpose
Migraine headaches usually cause a throbbing pain
on one side of the head. Nausea, vomiting, dizziness, and nausea are common symptoms.
The symptoms of migraine headaches are usually mild and usually disappear within a few days.
Symptoms of migraine headaches usually include:

Nausea

Nausea

Nausea

N

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: No context available
Question: No question available

Only return the helpful answer below and nothing else.
Helpful answer: No answer available

